# CNN

### Data가 상대적으로 적은 경우 cnn / 방대할 경우 transformer

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
#import global_device

In [18]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(32), # 32*32
        transforms.Normalize((0.5), (1.0)) # 평균 0.5 / 표준편차 1
    ]
)

train_data = datasets.MNIST(
    root='./',
    train=True,
    download=True,
    transform=data_transform # 전처리
)

test_data = datasets.MNIST(
    root='./',
    train=False,
    download=True,
    transform=data_transform # 전처리
) 

In [19]:
train_data.data.shape # data_loader로 불러오면 data_transform 설정으로 바뀜

torch.Size([60000, 28, 28])

In [20]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True) 
test_loader = DataLoader(test_data, batch_size=32) 
train_loader.dataset.data.shape

torch.Size([60000, 28, 28])

In [21]:
data,label = next(iter(train_loader))
print(data.shape) # 32로 사이즈 변화

torch.Size([32, 1, 32, 32])


In [22]:
class Lenet(nn.Module):
    
    def __init__(self):    
        super(Lenet, self).__init__() # 부모도 생성자에서 초기화
        self.conv1 = nn.Conv2d(
                                in_channels=1, 
                                out_channels=6, # 차원(장수)
                                kernel_size=5,  # 필터 사이즈
                                stride=1 
                            )
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1) 
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1) 
        
        self.fc1 = nn.Linear(in_features=120, out_features=84) # in_feature 계산해서 설정
        self.fc2 = nn.Linear(in_features=84, out_features=10) # out_feature 필요한 class로 설정
    
    def forward(self, x): # PyTorch의 예약어 함수        
        x = self.conv1(x) # 6, 28, 28   
        x = F.tanh(x) # 활성화 함수 
        x = F.max_pool2d(x, 2, 2) # 2*2로 pooling(mask) 진행 => 6, 14, 14
        
        x = self.conv2(x) # 16, 10, 10
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2) # 16, 5, 5
        
        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)
        
        x = self.fc1(x)
        x = F.tanh(x)
        
        x = self.fc2(x)
        x = F.tanh(x)
        
        return x # 10개의 값 
        
model = Lenet()        
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [23]:
from torchsummary import summary

summary(model, input_size=(1, 32, 32)) # 보고서에 total 학습 parameter 기재(1장 기준)

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0

In [24]:
custom_lr = 1e-3
epochs = 10

optim = Adam(model.parameters(), lr = custom_lr) # 최적화 함수
criterion = nn.CrossEntropyLoss() # 손실 함수

device = 'cuda' if torch.cuda.is_available else 'cpu'
model.to(device)

writer = SummaryWriter()

step = 0

for epoch in range(epochs):
    for data, label in train_loader:
        optim.zero_grad()
        pred = model(data.to(device)) # 32, 1, 32, 32
        loss = criterion(pred, label.to(device))
        writer.add_scalar('Loss/train', loss, step)
        step += 1
        
        loss.backward()
        optim.step()
        
    print(f'epoch: {epoch + 1}, loss: {loss.item()}')

epoch: 1, loss: 0.8461909294128418
epoch: 2, loss: 0.8857754468917847
epoch: 3, loss: 0.7976832389831543
epoch: 4, loss: 0.799486517906189
epoch: 5, loss: 0.7986149787902832
epoch: 6, loss: 0.8032175302505493
epoch: 7, loss: 0.809123694896698
epoch: 8, loss: 0.8059000372886658
epoch: 9, loss: 0.8732156157493591
epoch: 10, loss: 0.8033444285392761


## 모델 평가

In [25]:
model.eval() # 테스트 모드(추론)로 model을 전환(drop out등 값이 변화될 수 있는 것 제거)

with torch.no_grad(): # 자동 미분 종료
    
    total_corr = 0
    
    for images, labels in test_loader:
        images = images.to(device) # model과 input 일치화
        labels = labels.to(device)
        preds = model(images)
        _, pred = torch.max(preds.data, 1) # 적합도 배열, 1위 예측 배열
        
        total_corr = total_corr + (pred == labels).sum().item()

print(f'정확도 : {total_corr/len(test_data.targets)}')
print(preds.data.shape) # torch.Size([16, 10]) 32개씩 돌고 남은 16개

# 결과 확인 후 
# 1. Data 변경 2. Model 변경 3. Data 자체의 오류(ex: 9의 모양 확인)b

정확도 : 0.9878
torch.Size([16, 10])


## 모델 사용

In [51]:
from PIL import Image
import numpy as np

img = Image.open('data/5.jpg')
img = img.convert('L')  # grayscale로 변환
img = img.resize((32, 32))  # 크기 조절
img = np.array(img)  # numpy 배열로 변환
img = img.reshape(1, 32, 32)  # 차원 조절

img = torch.from_numpy(img).float()  # numpy 배열을 torch tensor로 변환
print(img.size())

model(img.to(device)) # torch.Size([1, 32, 32])

# preds = 
# _, pred = torch.max(preds.data, 1)

# print(torch.mode(pred).values.item())

torch.Size([1, 32, 32])


tensor([[-1.0000, -1.0000, -1.0000,  0.9894, -1.0000, -0.9992, -1.0000, -1.0000,
          0.1001, -0.9909]], device='cuda:0', grad_fn=<TanhBackward0>)

In [52]:
from PIL import Image
import numpy as np

img = Image.open('data/1.jpg')
img_array = np.array(img)
img_array.shape

(579, 576, 3)

In [53]:
infer_transform = transforms.Compose( # 전처리
    [
        transforms.ToTensor(),
        transforms.Grayscale(), # 흑백으로 변경(1차원 변경)
        transforms.Resize(32), # 32*32
        transforms.Normalize((0.5), (1.0)) # 학습 데이터도 정규화 했으므로 입력값도 정규화
    ]
)

infer_img = infer_transform(img)
infer_img.size()

torch.Size([1, 32, 32])

In [54]:
model(infer_img.to(device))

tensor([[-0.9551,  1.0000, -1.0000, -1.0000, -0.9999,  0.9994,  0.6047, -0.0615,
         -1.0000, -1.0000]], device='cuda:0', grad_fn=<TanhBackward0>)